In [17]:
# Initial imports
import os
import requests
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import hvplot.pandas
load_dotenv()
from pathlib import Path
from datetime import datetime, timedelta

In [18]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [19]:
# Set start and end datetimes of 7 days back from from Today
end_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
start_date = pd.Timestamp(datetime.now()- timedelta(7), tz="America/New_York").isoformat()
print(f'Start date: {start_date} End date: {end_date}')

Start date: 2021-11-23T09:52:42.954447-05:00 End date: 2021-11-30T09:52:42.954447-05:00


In [20]:
# Set the Tesla, Apple, Netflix,Coca-Cola and General Electric tickers
tickers = ["TSLA", "AAPL", "NFLX", "KO", "GE"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for TSLA and KO
alpaca_prices = api.get_barset(tickers, timeframe,start = start_date, end = end_date, limit = 1000).df
alpaca_prices.dropna(inplace = True)
alpaca_prices.isnull().sum()

AAPL  open      0
      high      0
      low       0
      close     0
      volume    0
GE    open      0
      high      0
      low       0
      close     0
      volume    0
KO    open      0
      high      0
      low       0
      close     0
      volume    0
NFLX  open      0
      high      0
      low       0
      close     0
      volume    0
TSLA  open      0
      high      0
      low       0
      close     0
      volume    0
dtype: int64

In [21]:
# Display prices
alpaca_prices

AAPL                                      \
                              open    high       low   close    volume   
time                                                                     
2021-11-23 00:00:00-05:00  161.120  161.80  159.0601  161.42  85191018   
2021-11-24 00:00:00-05:00  160.750  162.14  159.6400  161.94  61311962   
2021-11-26 00:00:00-05:00  159.565  160.45  156.3600  156.88  63333097   
2021-11-29 00:00:00-05:00  159.370  161.19  158.7901  160.23  76825307   

                               GE                                    ...  \
                             open     high     low   close   volume  ...   
time                                                                 ...   
2021-11-23 00:00:00-05:00  101.78  102.210  101.15  102.08  5568872  ...   
2021-11-24 00:00:00-05:00  102.05  102.315  101.31  102.27  4134897  ...   
2021-11-26 00:00:00-05:00   97.00   98.100   95.51   97.77  7790657  ...   
2021-11-29 00:00:00-05:00   99.27  100.430   96.83   98.35  7973905  ...   

                             NFLX                                       TSLA  \
                             open     high     low   close   volume     open   
time                                                                           
2021-11-23 00:00:00-05:00  658.11  666.430  646.05  654.00  2000072  1167.51   
2021-11-24 00:00:00-05:00  657.84  661.440  651.10  658.19  1627030  1080.39   
2021-11-26 00:00:00-05:00  675.15  676.405  660.67  665.60  2443953  1099.44   
2021-11-29 00:00:00-05:00  664.41  667.990  658.29  663.68  2134362  1100.79   

                                                                  
                                high      low    close    volume  
time                                                              
2021-11-23 00:00:00-05:00  1180.4999  1062.70  1108.81  34397054  
2021-11-24 00:00:00-05:00  1132.7700  1062.00  1115.85  20960960  
2021-11-26 00:00:00-05:00  1108.7827  1081.00  1081.79  10205161  
2021-11-29 00:00:00-05:00  1142.6700  1100.19  1136.98  17659389  

[4 rows x 25 columns]

In [22]:
# Store only closing prices
alpaca_closing_prices = pd.DataFrame()
for ticker in tickers:
    alpaca_closing_prices[ticker]= alpaca_prices[ticker]['close']
# Preview DataFrame after dropping nulls and time
alpaca_closing_prices.dropna(inplace = True)
alpaca_closing_prices.isnull().sum()

TSLA    0
AAPL    0
NFLX    0
KO      0
GE      0
dtype: int64

In [23]:
# Display only closing prices
alpaca_closing_prices

,TSLA,AAPL,NFLX,KO,GE
time,,,,,
2021-11-23 00:00:00-05:00,1108.81,161.42,654.00,55.88,102.08
2021-11-24 00:00:00-05:00,1115.85,161.94,658.19,55.43,102.27
2021-11-26 00:00:00-05:00,1081.79,156.88,665.60,53.77,97.77
2021-11-29 00:00:00-05:00,1136.98,160.23,663.68,54.57,98.35


In [24]:
# Use the `pct_change` function to calculate daily return
alpaca_returns = alpaca_closing_prices.pct_change().dropna()
alpaca_returns

,TSLA,AAPL,NFLX,KO,GE
time,,,,,
2021-11-24 00:00:00-05:00,0.006349,0.003221,0.006407,-0.008053,0.001861
2021-11-26 00:00:00-05:00,-0.030524,-0.031246,0.011258,-0.029948,-0.044001
2021-11-29 00:00:00-05:00,0.051017,0.021354,-0.002885,0.014878,0.005932


In [25]:
# Plot returns
alpaca_returns.hvplot.line(title = "Percentage Change- Returns", xlabel = "Change", ylabel = "Date")

:NdOverlay   [Variable]
   :Curve   [time]   (value)

In [26]:
# Save pre-processed Closing Prices from Alpaca
alpaca_prices.to_csv(Path('c:/Users/User/Documents/Renu/Project2/Data/alpaca_prices.csv'))
alpaca_returns.to_csv(Path('c:/Users/User/Documents/Renu/Project2/Data/alpaca_returns.csv'))
alpaca_closing_prices.to_csv(Path('c:/Users/User/Documents/Renu/Project2/Data/alpaca_closing_prices.csv'))

In [27]:
# The End